In [1]:
from langchain_openai import OpenAI,ChatOpenAI
from langchain_community.chat_models import QianfanChatEndpoint
from langchain_core.language_models.chat_models import HumanMessage
import os
from langchain_community.chat_models.tongyi import ChatTongyi

In [2]:
chat_model = ChatOpenAI(temperature=0,model_name="gpt-3.5-turbo")
qian_fan_model = QianfanChatEndpoint(streaming=True,temperature=0.1)
chat_tongyi = ChatTongyi(streaming=True,temperature=0.1)


In [3]:
from langchain.schema  import SystemMessage, HumanMessage, AIMessage

In [4]:
messages =[
    SystemMessage(content='You are a helpful assistant.'),
    HumanMessage(content="Who won the world series in 2020?"),
    AIMessage(content="The Los Angeles Dodgers won the World Series in 2020."), 
   HumanMessage(content="Where was it played?")
]

In [5]:
messages

[SystemMessage(content='You are a helpful assistant.'),
 HumanMessage(content='Who won the world series in 2020?'),
 AIMessage(content='The Los Angeles Dodgers won the World Series in 2020.'),
 HumanMessage(content='Where was it played?')]

In [91]:
response = chat_model.invoke("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

In [90]:
qian_fan_model.invoke("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

[WARNING] [04-16 18:18:13] base.py:406 [t:6916]: retry is not available when stream is enabled
[WARNING] [04-16 18:18:13] base.py:621 [t:6916]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept


AIMessage(content='lollipopopolliloollllo\n\nThe letters in l-o-l-l-i-p-o-p are rearranged and reversed to produce the resulting word or phrase "lollipop".', response_metadata={'token_usage': {}, 'model_name': 'ERNIE-Bot-turbo', 'finish_reason': 'stop'}, id='run-961f7455-798b-49a1-82b2-9bc76c482fdd-0')

In [8]:
chat_tongyi.invoke(messages)

AIMessage(content='Due to the COVID-19 pandemic, the 2020 World Series was held at a unique location without a home field advantage for either team. The games were played at two neutral sites: Dodger Stadium in Los Angeles, California, for the first two games and the American Airlines Center in Dallas, Texas, for the remaining games. The series took place from October 24 to October 28, 2020.', response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '4e373ce7-38ba-9d5d-9192-df4410d262f2', 'token_usage': {'input_tokens': 61, 'output_tokens': 91, 'total_tokens': 152}}, id='run-eae7643a-5906-46fe-9239-f2394df4a90e-0')

In [92]:
print(response)

content='p-o-p-i-l-l-o-l' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 22, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None} id='run-4cb3a564-3a30-4ae7-92f8-7e54859e5614-0'


In [10]:
from langchain.prompts import (PromptTemplate,HumanMessagePromptTemplate,ChatPromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate)

In [11]:
# 翻译任务指令始终由 System 角色承担
template = (
    """You are a translation expert, proficient in various languages. \n
    Translates English to Chinese."""
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [12]:
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a translation expert, proficient in various languages. \n\n    Translates English to Chinese.'))

In [13]:
# 待翻译文本由 Human 角色输入
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [14]:
human_message_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))

In [15]:
# 使用 System 和 Human 角色的提示模板构造 ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [16]:
print(chat_prompt_template)

input_variables=['text'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a translation expert, proficient in various languages. \n\n    Translates English to Chinese.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]


In [17]:
# 生成用于翻译的 Chat Prompt
chat_prompt_template.format_prompt(text="I love programming.")

ChatPromptValue(messages=[SystemMessage(content='You are a translation expert, proficient in various languages. \n\n    Translates English to Chinese.'), HumanMessage(content='I love programming.')])

In [18]:
chat_prompt = chat_prompt_template.format_prompt(text="I love programming.").to_messages()

In [19]:
chat_prompt

[SystemMessage(content='You are a translation expert, proficient in various languages. \n\n    Translates English to Chinese.'),
 HumanMessage(content='I love programming.')]

In [20]:
type(chat_prompt)

list

In [21]:
chat_prompt = chat_prompt_template.format_messages(text='I love programming.')

In [22]:
chat_prompt

[SystemMessage(content='You are a translation expert, proficient in various languages. \n\n    Translates English to Chinese.'),
 HumanMessage(content='I love programming.')]

In [23]:
type(chat_prompt)

list

In [24]:
chat_model.invoke(chat_prompt)

AIMessage(content='我喜欢编程。', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 34, 'total_tokens': 42}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e93bc03-def8-474c-8200-d5391bccee2e-0')

In [25]:
from langchain.chains import ConversationChain,LLMChain
from langchain.memory import ConversationBufferMemory

In [26]:
 #无需再每次都使用 to_messages 方法构造 Chat Prompt
translation_chain  = LLMChain(llm=chat_model,prompt=chat_prompt_template,verbose=True)

In [27]:
qian_fan_translation_chain = LLMChain(llm=qian_fan_model,prompt=chat_prompt_template,verbose=True)

In [28]:
tongyi_translation_chain = LLMChain(llm=chat_tongyi,prompt= chat_prompt_template,verbose=True)

In [29]:
translation_chain.invoke({
    "text": "I love programming."
})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a translation expert, proficient in various languages. 

    Translates English to Chinese.
Human: I love programming.

> Finished chain.


{'text': '我喜欢编程。'}

In [30]:
qian_fan_translation_chain.invoke({
    "text": "I love programming."
})

[WARNING] [04-16 17:08:21] base.py:406 [t:6916]: retry is not available when stream is enabled
[WARNING] [04-16 17:08:21] base.py:621 [t:6916]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a translation expert, proficient in various languages. 

    Translates English to Chinese.
Human: I love programming.

> Finished chain.


{'text': '编程是一项非常有趣和有挑战性的工作，它需要大量的逻辑思维和解决问题的能力。很高兴听到你对编程感兴趣。你最喜欢编程的哪个方面呢？'}

In [31]:
tongyi_translation_chain.invoke({
    "text": "I love programming."
})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a translation expert, proficient in various languages. 

    Translates English to Chinese.
Human: I love programming.

> Finished chain.


{'text': '我热爱编程。'}

In [32]:
translation_chain.invoke({'text': "I love AI and Large Language Model."})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a translation expert, proficient in various languages. 

    Translates English to Chinese.
Human: I love AI and Large Language Model.

> Finished chain.


{'text': '我喜欢人工智能和大型语言模型。'}

In [33]:
qian_fan_translation_chain.invoke({'text': "I love AI and Large Language Model."})

[WARNING] [04-16 17:08:24] base.py:406 [t:6916]: retry is not available when stream is enabled
[WARNING] [04-16 17:08:24] base.py:621 [t:6916]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a translation expert, proficient in various languages. 

    Translates English to Chinese.
Human: I love AI and Large Language Model.

> Finished chain.


{'text': 'I am glad to hear that you love AI and Large Language Model. AI and Large Language Model are becoming increasingly important in modern society, and they have made great progress in various fields. I hope you can continue to explore and learn more about AI and Large Language Model, and share your experiences with me.'}

In [34]:
translation_chain.invoke({'text': "['Fruit', 'Color', 'Price (USD)'] [Apple, Red, 1.20], [Banana, Yellow, 0.50] [Orange, Orange, 0.80] [Strawberry, Red, 2.50] [Blueberry, Blue, 3.00] [Kiwi, Green, 1.00] [Mango, Orange, 1.50] [Grape, Purple, 2.00]"})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a translation expert, proficient in various languages. 

    Translates English to Chinese.
Human: ['Fruit', 'Color', 'Price (USD)'] [Apple, Red, 1.20], [Banana, Yellow, 0.50] [Orange, Orange, 0.80] [Strawberry, Red, 2.50] [Blueberry, Blue, 3.00] [Kiwi, Green, 1.00] [Mango, Orange, 1.50] [Grape, Purple, 2.00]

> Finished chain.


{'text': "['水果', '颜色', '价格（美元）'] [苹果，红色，1.20] [香蕉，黄色，0.50] [橙子，橙色，0.80] [草莓，红色，2.50] [蓝莓，蓝色，3.00] [猕猴桃，绿色，1.00] [芒果，橙色，1.50] [葡萄，紫色，2.00]"}

In [35]:
# 翻译任务指令始终由 System 角色承担
template = (
    """You are a translation expert, proficient in various languages. \n
    Translates {source_language} to {target_language}."""
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template) 

In [36]:
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['source_language', 'target_language'], template='You are a translation expert, proficient in various languages. \n\n    Translates {source_language} to {target_language}.'))

In [37]:
# 待翻译文本由 Human 角色输入
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [38]:
# 使用 System 和 Human 角色的提示模板构造 ChatPromptTemplate
m_chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [77]:
m_translation_chain = LLMChain(llm=chat_model, verbose=True,prompt=m_chat_prompt_template)

In [76]:
m_translation_chain.invoke({
    "source_language": "Chinese",
    "target_language": "English",
    "text": "我喜欢学习大语言模型，轻松简单又愉快",
})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a translation expert, proficient in various languages. 

    Translates Chinese to English.
Human: 我喜欢学习大语言模型，轻松简单又愉快

> Finished chain.


{'source_language': 'Chinese',
 'target_language': 'English',
 'text': 'I enjoy studying large language models, which are easy, simple, and enjoyable.'}

In [84]:
from typing import Dict, List

from langchain_core.memory import BaseMemory
from langchain_core.prompts import BasePromptTemplate
from langchain_core.pydantic_v1 import Extra, Field, root_validator

from langchain.chains.conversation.prompt import PROMPT
from langchain.chains.llm import LLMChain
from langchain.memory.buffer import ConversationBufferMemory
from langchain_core.runnables import (
    RunnableConfig,
    RunnableSerializable,
    ensure_config,
    run_in_executor,
)
from typing import Any, Dict, List, Optional, Type, Union, cast

class TranslationChain(LLMChain):
    prompt: BasePromptTemplate 
    
    output_key: str = "response"  #: :meta private:

    class Config:
        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]
    
    def invoke(
        self,
        input: Dict[str, Any],
        config: Optional[RunnableConfig] = None,
        **kwargs: Any,
    ) -> Dict[str, Any]:
        if 'source_language' not in input or input['source_language'] is None:
            input['source_language'] = 'Chinese'
        if 'target_language' not in input or input['target_language'] is None:
            input['target_language'] = 'English'
        return super().invoke(input, config, **kwargs)        

     
    

In [85]:
c_translation_chain = TranslationChain(llm=chat_model, verbose=True,prompt=m_chat_prompt_template)

In [86]:
c_translation_chain.invoke(input={
    "text": "I enjoy studying large language models, it's easy, simple, and enjoyable.",
    "source_language":'English',
    "target_language":'Chinese',
})



> Entering new TranslationChain chain...
Prompt after formatting:
System: You are a translation expert, proficient in various languages. 

    Translates English to Chinese.
Human: I enjoy studying large language models, it's easy, simple, and enjoyable.

> Finished chain.


{'text': "I enjoy studying large language models, it's easy, simple, and enjoyable.",
 'source_language': 'English',
 'target_language': 'Chinese',
 'response': '我喜欢研究大型语言模型，这很容易、简单且令人愉悦。'}